In [1]:
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from numpy.random import randint
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Sequential
from keras.layers.core import Flatten, Dense
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import SGD
import matplotlib.pylab as plt
import numpy as np
from keras.utils import np_utils

import cv2
import h5py
import os
import h5py

Using Theano backend.
Using gpu device 0: GeForce GTX 950M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5005)


In [2]:
import boilerplate
from boilerplate import keras_resized_code
reload(keras_resized_code)

<module 'boilerplate.keras_resized_code' from 'boilerplate\keras_resized_code.pyc'>

In [ ]:
# read from h5py
with h5py.File('data.h5', 'r') as hf:
    print('List of arrays in this file: \n', hf.keys())
    data = hf.get('X_train')
    X_train = np.array(data)
    data = hf.get('y_train')
    y_train = np.array(data)
    data = hf.get('X_test')
    X_test = np.array(data)
    data = hf.get('y_test')
    y_test = np.array(data)    

In [ ]:
X_train, y_train = keras_resized_code.load_train_resized('resized/')

0it [00:00, ?it/s]
  1%|▊                                                                                 | 4/403 [00:00<00:12, 32.79it/s]

In [4]:
y_train = np_utils.to_categorical(y_train,10)
#y_test = np_utils.to_categorical(y_test,10)

In [5]:
X_train = np.transpose(X_train, (0,3,1,2))

In [6]:
X_train[0].shape

(3L, 224L, 224L)

In [7]:
def vgg_std16_model(img_rows, img_cols, color_type=3):
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(color_type,
                                                 img_rows, img_cols)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    model.load_weights('pretrained_models/vgg16_weights.h5')

    # Code above loads pre-trained data and
    model.layers.pop()
    model.add(Dense(10, activation='softmax'))
    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
model = vgg_std16_model(224,224)

In [ ]:
model.fit(X_train,y_train,batch_size=4,nb_epoch=5)

Epoch 1/5
320/606 [==============>...............] - ETA: 66s - loss: 2.3013 - acc: 0.1375

In [ ]:
model.save_weights('vgg16_1epoch')